In [1]:
import numpy as np 
import pandas as pd 

In [2]:
df = pd.read_csv('../input/fake-news/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
# Creating the feature matrix
X = df.drop('label', axis = 1)

In [6]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [7]:
# Get the dependent faeatures
y = df['label']
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [8]:
# Removing the NaN values
df = df.dropna()

In [9]:
df.shape

(18285, 5)

In [10]:
# Importing the libraries
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [11]:
# Vocabulary size
vocab_size = 5000

### One-Hot Representation

In [12]:
messages = df.copy()

In [13]:
# Resetting the indices after the removal of NaN values
messages.reset_index(inplace = True)

In [14]:
messages['text'][4]

'Print \nAn Iranian woman has been sentenced to six years in prison after Iran’s Revolutionary Guard searched her home and found a notebook that contained a fictional story she’d written about a woman who was stoned to death, according to the Eurasia Review . \nGolrokh Ebrahimi Iraee, 35, is the wife of political prisoner Arash Sadeghi, 36, who is serving a 19-year prison sentence for being a human rights activist, the publication reported. \n“When the intelligence unit of the Revolutionary Guards came to arrest her husband, they raided their apartment – without a warrant – and found drafts of stories that Ebrahimi Iraee had written,” the article stated. \n“One of the confiscated drafts was a story about stoning women to death for adultery – never published, never presented to anyone,” the article stated. “The narrative followed the story of a protagonist that watched a movie about stoning of women under Islamic law for adultery.'

*Data Pre-processing*

In [15]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [17]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [18]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [19]:
onehot_rep = [one_hot(words,vocab_size) for words in corpus]
onehot_rep

[[2052, 4972, 3963, 2566, 2594, 436, 4378, 1793, 3641, 4454],
 [2719, 2927, 1828, 2797, 3653, 1997, 1526],
 [4943, 1150, 94, 3472],
 [2152, 70, 66, 3868, 447, 1395],
 [4365, 3653, 672, 950, 4830, 1125, 3653, 1736, 2386, 1956],
 [2601,
  2716,
  299,
  119,
  1341,
  1552,
  1751,
  4929,
  3480,
  2606,
  2727,
  3383,
  1433,
  2773,
  1526],
 [1658, 1518, 4195, 3417, 3913, 831, 2169, 3246, 4012, 2679, 3932],
 [2139, 2495, 2084, 2644, 671, 17, 1552, 4725, 4012, 2679, 3932],
 [3391, 3052, 3511, 778, 647, 1595, 3640, 3, 1552, 2670],
 [3524, 1147, 2735, 2833, 3212, 4821, 4936, 1869],
 [2288, 2348, 3145, 3390, 3262, 206, 2974, 2145, 3001, 4734, 2607],
 [3868, 4838, 2594, 1595, 1552, 671],
 [4899, 1138, 1391, 1632, 339, 2740, 357, 3026, 610],
 [3455, 2131, 1579, 3381, 3465, 4161, 309, 4012, 2679, 3932],
 [4585, 2593, 507, 4627, 1921, 4012, 2679, 3932],
 [628, 1193, 1659, 4872, 2671, 383, 133, 3149, 3868, 2499],
 [921, 2911, 2927],
 [2992, 3041, 4700, 1835, 1552, 3169, 3141, 1526],
 [843, 2

### Embedding Representation

In [20]:
sent_len = 20
embedded = pad_sequences(onehot_rep, padding = 'pre', maxlen = sent_len)
print(embedded)

[[   0    0    0 ... 1793 3641 4454]
 [   0    0    0 ... 3653 1997 1526]
 [   0    0    0 ... 1150   94 3472]
 ...
 [   0    0    0 ... 4012 2679 3932]
 [   0    0    0 ...  871 2219 1005]
 [   0    0    0 ... 2448 2842 2888]]


In [21]:
len(embedded)

18285

In [22]:
embedded[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2052,
       4972, 3963, 2566, 2594,  436, 4378, 1793, 3641, 4454], dtype=int32)

### Creating the Model

In [23]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length = sent_len))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
y = messages['label']

In [25]:
import numpy as np
X_final = np.array(embedded)
y_final = np.array(y)

In [26]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [27]:
# Train-Test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=30)

In [28]:
# Train the model
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=30,batch_size=64)

Epoch 1/30
192/192 [==============================] - 2s 10ms/step - loss: 0.3251 - accuracy: 0.8538 - val_loss: 0.1938 - val_accuracy: 0.9188
Epoch 2/30
192/192 [==============================] - 1s 8ms/step - loss: 0.1375 - accuracy: 0.9463 - val_loss: 0.1983 - val_accuracy: 0.9135
Epoch 3/30
192/192 [==============================] - 1s 8ms/step - loss: 0.1006 - accuracy: 0.9639 - val_loss: 0.2049 - val_accuracy: 0.9138
Epoch 4/30
192/192 [==============================] - 1s 8ms/step - loss: 0.0752 - accuracy: 0.9747 - val_loss: 0.2289 - val_accuracy: 0.9223
Epoch 5/30
192/192 [==============================] - 1s 8ms/step - loss: 0.0555 - accuracy: 0.9810 - val_loss: 0.2726 - val_accuracy: 0.9193
Epoch 6/30
192/192 [==============================] - 2s 8ms/step - loss: 0.0432 - accuracy: 0.9855 - val_loss: 0.2583 - val_accuracy: 0.9167
Epoch 7/30
192/192 [==============================] - 2s 8ms/step - loss: 0.0308 - accuracy: 0.9905 - val_loss: 0.3619 - val_accuracy: 0.9178
Epoch

### Performance Metrics

In [29]:
y_pred = (model.predict_classes(X_test))

In [30]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
confusion_matrix(y_test,y_pred)
accuracy_score(y_test,y_pred)

0.9163214581607291

### Adding Dropout

In [40]:
from tensorflow.keras.layers import Dropout
# Creating the model
embedding_vector_features = 50
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length = sent_len))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 50)            250000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________
None


In [49]:
X_final = np.array(embedded)
y_final = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=0)

# Train the model
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=30,batch_size=128)

Epoch 1/30
100/100 [==============================] - 1s 9ms/step - loss: 0.0549 - accuracy: 0.9868 - val_loss: 0.0492 - val_accuracy: 0.9880
Epoch 2/30
100/100 [==============================] - 1s 9ms/step - loss: 0.0183 - accuracy: 0.9955 - val_loss: 0.0586 - val_accuracy: 0.9872
Epoch 3/30
100/100 [==============================] - 1s 8ms/step - loss: 0.0125 - accuracy: 0.9968 - val_loss: 0.0578 - val_accuracy: 0.9876
Epoch 4/30
100/100 [==============================] - 1s 8ms/step - loss: 0.0076 - accuracy: 0.9981 - val_loss: 0.0650 - val_accuracy: 0.9867
Epoch 5/30
100/100 [==============================] - 1s 9ms/step - loss: 0.0048 - accuracy: 0.9988 - val_loss: 0.0677 - val_accuracy: 0.9869
Epoch 6/30
100/100 [==============================] - 1s 10ms/step - loss: 0.0033 - accuracy: 0.9991 - val_loss: 0.0753 - val_accuracy: 0.9867
Epoch 7/30
100/100 [==============================] - 1s 9ms/step - loss: 0.0028 - accuracy: 0.9990 - val_loss: 0.0738 - val_accuracy: 0.9865
Epoch

In [58]:
y_pred = (model.predict_classes(X_test))
print('\n Confusion Matrix:- \n',confusion_matrix(y_test,y_pred))
print('\n Accuracy:',accuracy_score(y_test,y_pred)* 100,'%')


 Confusion Matrix:- 
 [[3022   62]
 [  44 2358]]

 Accuracy: 98.0678089682829 %
